# Manipulação de Datas e Timestamps no Apache Spark

O Apache Spark oferece uma variedade de funções para manipular datas e timestamps em DataFrames. Essas funções são úteis para realizar operações de formatação, cálculos de diferença de tempo, extração de componentes de datas e muito mais.

## Funções de Manipulação de Datas e Timestamps

As funções a seguir são comumente usadas para manipulação de datas e timestamps:

| Função                            | Descrição                                                      | Exemplo de Caso de Uso                            |
|-----------------------------------|----------------------------------------------------------------|---------------------------------------------------|
| `current_date()`                  | Retorna a data atual                                           | Marcar datas de registro.                         |
| `current_timestamp()`             | Retorna a data e hora atual                                    | Registrar timestamps de eventos.                  |
| `date_add(start_date, days)`      | Adiciona um número de dias a uma data                           | Prever datas futuras com base em eventos passados. |
| `date_sub(start_date, days)`      | Subtrai um número de dias de uma data                           | Calcular datas retroativas.                       |
| `datediff(end_date, start_date)`  | Calcula o número de dias entre duas datas                      | Medir a diferença entre datas.                     |
| `months_between(end_date, start_date, roundOff)` | Calcula o número de meses entre duas datas | Análise de tendências e sazonalidade. |
| `year(date)`                      | Extrai o ano de uma data ou timestamp                          | Agrupar dados por ano.                            |
| `quarter(date)`                   | Extrai o trimestre de uma data ou timestamp                    | Agrupar dados por trimestre.                      |
| `month(date)`                     | Extrai o mês de uma data ou timestamp                          | Agrupar dados por mês.                            |
| `dayofmonth(date)`                | Extrai o dia do mês de uma data ou timestamp                   | Analisar padrões de comportamento mensal.          |
| `dayofweek(date)`                 | Extrai o dia da semana de uma data ou timestamp                 | Analisar padrões semanais.                         |
| `hour(timestamp)`                 | Extrai a hora de um timestamp                                   | Analisar padrões de atividade por hora.             |
| `minute(timestamp)`               | Extrai os minutos de um timestamp                              | Analisar padrões de atividade por minuto.           |
| `second(timestamp)`               | Extrai os segundos de um timestamp                              | Analisar padrões de atividade por segundo.          |


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Exemplo").config("spark.jars.packages", "org.postgresql:postgresql:42.2.24").getOrCreate()

In [2]:
df_vra = (
    spark
    .read
    .format("parquet")
    .load("/home/app/data/1.bronze/vra/")
)
df_vra.show(1, vertical=True)

-RECORD 0----------------------------------
 icao_empresa_aerea     | TAM              
 numero_voo             | 9458             
 codigo_di              | 0                
 codigo_tipo_linha      | I                
 icao_aerodromo_origem  | SBGR             
 icao_aerodromo_destino | SCEL             
 partida_prevista       | 01/01/2020 06:30 
 partida_real           | 01/01/2020 06:30 
 chegada_prevista       | 01/01/2020 10:40 
 chegada_real           | 01/01/2020 10:40 
 situacao_voo           | REALIZADO        
 codigo_justificativa   | null             
only showing top 1 row



In [15]:
import pyspark.sql.functions as F

(
    df_vra
    .withColumn("partida_prevista", F.to_timestamp("partida_prevista",'dd/MM/yyyy HH:mm'))
    .withColumn("partida_real", F.to_timestamp("partida_real",'dd/MM/yyyy HH:mm'))
    .withColumn("chegada_prevista", F.to_timestamp("chegada_prevista",'dd/MM/yyyy HH:mm'))
    .withColumn("chegada_real", F.to_timestamp("chegada_real",'dd/MM/yyyy HH:mm'))
    .withColumn("tempo_de_voo_real", F.datediff("chegada_real","partida_real"))
    .withColumn("tempo_de_voo_prevista", F.datediff("chegada_prevista","partida_prevista"))
    .show(1, vertical=True)
)

-RECORD 0-------------------------------------
 icao_empresa_aerea     | TAM                 
 numero_voo             | 9458                
 codigo_di              | 0                   
 codigo_tipo_linha      | I                   
 icao_aerodromo_origem  | SBGR                
 icao_aerodromo_destino | SCEL                
 partida_prevista       | 2020-01-01 06:30:00 
 partida_real           | 2020-01-01 06:30:00 
 chegada_prevista       | 2020-01-01 10:40:00 
 chegada_real           | 2020-01-01 10:40:00 
 situacao_voo           | REALIZADO           
 codigo_justificativa   | null                
 tempo_de_voo_real      | 0                   
 tempo_de_voo_prevista  | 0                   
only showing top 1 row



In [5]:
df_cia = (
    spark
    .read
    .format("parquet")
    .load("data/1.bronze/air_cia/")
)
df_cia.show(1, vertical=True)

-RECORD 0----------------------------------------
 razão_social             | ABSA - AEROLINHAS... 
 icao_iata                | LTG M3               
 cnpj                     | 00074635000133       
 atividades_aéreas        | TRANSPORTE AÉREO ... 
 endereço_sede            | AEROPORTO INTERNA... 
 telefone                 | 1155828055           
 e-mail                   | gar@tam.com.br       
 decisão_operacional      | DECISÃO Nº 41        
 data_decisão_operacional | 22/04/2015           
 validade_operacional     | 23/04/2025           
 icao                     | LTG                  
 iata                     | M3                   
only showing top 1 row



In [20]:
(
    df_cia
    .withColumn("data_decisão_operacional" , F.to_date(F.col('data_decisão_operacional'),'dd/MM/yyyy'))
    .withColumn("validade_operacional" , F.to_date(F.col('validade_operacional'),'dd/MM/yyyy'))
    .withColumn("year", F.year(F.col('validade_operacional')))
    .withColumn("month", F.month(F.col('validade_operacional')))
    .withColumn("quarter", F.quarter(F.col('validade_operacional')))
    .withColumn("day_of_week", F.dayofweek(F.col('validade_operacional')))
    .withColumn("day_of_month", F.dayofmonth(F.col('validade_operacional')))
    .withColumn("day", F.day(F.col('validade_operacional')))
    .show(1, vertical=True)
)

AttributeError: module 'pyspark.sql.functions' has no attribute 'day'